In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import day_abbr, month_abbr, mdays
import datetime
import holidays



In [9]:
df =  pd.read_csv("CleanHHElec.csv", index_col = 0, parse_dates=True)
df

,Target,n-1,n-2,n-3,n-4,n-5
From Timestamp,,,,,,
2018-01-01,12.60,12.80,11.90,13.50,12.30,12.80
2018-01-01,11.90,12.60,12.80,11.90,13.50,12.30
2018-01-01,13.00,11.90,12.60,12.80,11.90,13.50
2018-01-01,11.91,13.00,11.90,12.60,12.80,11.90
2018-01-01,11.90,11.91,13.00,11.90,12.60,12.80
...,...,...,...,...,...,...
2019-12-31,13.56,15.12,15.00,14.56,15.00,15.31
2019-12-31,13.81,13.56,15.12,15.00,14.56,15.00
2019-12-31,14.12,13.81,13.56,15.12,15.00,14.56


In [10]:
df['dayofweek'] = df.index.dayofweek
df['hour'] = df.index.hour
#df['minute'] = df.index.minute
#df.loc[df['minute'] == 30, 'hour'] += 0.5
#df.drop('minute', axis=1, inplace=True)
df['holiday'] = df.index.minute #naming it holiday for later use as minutes column would be dropped anyway
df.loc[df['holiday'] == 30, 'hour'] += 0.5
df.loc[df['holiday'] == 30, 'holiday'] = 0
df['month'] = df.index.month
df['Date'] = df.index.date

df


,Target,n-1,n-2,n-3,n-4,n-5,dayofweek,hour,holiday,month,Date
From Timestamp,,,,,,,,,,,
2018-01-01,12.60,12.80,11.90,13.50,12.30,12.80,0,0.0,0,1,2018-01-01
2018-01-01,11.90,12.60,12.80,11.90,13.50,12.30,0,0.0,0,1,2018-01-01
2018-01-01,13.00,11.90,12.60,12.80,11.90,13.50,0,0.0,0,1,2018-01-01
2018-01-01,11.91,13.00,11.90,12.60,12.80,11.90,0,0.0,0,1,2018-01-01
2018-01-01,11.90,11.91,13.00,11.90,12.60,12.80,0,0.0,0,1,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,13.56,15.12,15.00,14.56,15.00,15.31,1,0.0,0,12,2019-12-31
2019-12-31,13.81,13.56,15.12,15.00,14.56,15.00,1,0.0,0,12,2019-12-31
2019-12-31,14.12,13.81,13.56,15.12,15.00,14.56,1,0.0,0,12,2019-12-31


# Adding Holiday Data 
Making holiday = 1 wherever that day was a holiday

In [11]:
for holidaydate, name in sorted(holidays.England(years=np.arange(2018, 2019 + 1)).items()):
    df.loc[df['Date'] == holidaydate, 'holiday'] = 1
    
df.drop('Date', axis=1, inplace=True)

df[df['holiday'] == 1]




,Target,n-1,n-2,n-3,n-4,n-5,dayofweek,hour,holiday,month
From Timestamp,,,,,,,,,,
2018-01-01,12.60,12.80,11.90,13.50,12.30,12.80,0,0.0,1,1
2018-01-01,11.90,12.60,12.80,11.90,13.50,12.30,0,0.0,1,1
2018-01-01,13.00,11.90,12.60,12.80,11.90,13.50,0,0.0,1,1
2018-01-01,11.91,13.00,11.90,12.60,12.80,11.90,0,0.0,1,1
2018-01-01,11.90,11.91,13.00,11.90,12.60,12.80,0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...
2019-12-26,15.50,16.00,15.88,15.88,15.31,15.62,3,0.0,1,12
2019-12-26,15.25,15.50,16.00,15.88,15.88,15.31,3,0.0,1,12
2019-12-26,15.00,15.25,15.50,16.00,15.88,15.88,3,0.0,1,12


# Adding Weather Data

In [12]:
weather_df = pd.read_excel("DoubleWeather.xlsx", usecols = [0,3,4,6], parse_dates=True)
weather_df = weather_df.rename(columns={weather_df.columns[0]: 'id',
                                       weather_df.columns[1]: 'Temperature (C)',
                                       weather_df.columns[2]: 'Feels like (C)',
                                       weather_df.columns[3]: 'Humidity (%)'})
weather_df=weather_df.set_index(['id'])
#weather_df.index = pd.to_datetime(weather_df.index)
weather_df

,Temperature (C),Feels like (C),Humidity (%)
id,,,
0,6.4,1.9,82.76
1,6.4,1.9,82.76
2,6.6,2.4,80.14
3,6.6,2.4,80.14
4,6.1,2.0,84.33
...,...,...,...
35035,6.5,3.3,88.23
35036,6.6,4.4,87.69
35037,6.6,4.4,87.69


In [13]:
df = df.reset_index()
df

,From Timestamp,Target,n-1,n-2,n-3,n-4,n-5,dayofweek,hour,holiday,month
0,2018-01-01,12.60,12.80,11.90,13.50,12.30,12.80,0,0.0,1,1
1,2018-01-01,11.90,12.60,12.80,11.90,13.50,12.30,0,0.0,1,1
2,2018-01-01,13.00,11.90,12.60,12.80,11.90,13.50,0,0.0,1,1
3,2018-01-01,11.91,13.00,11.90,12.60,12.80,11.90,0,0.0,1,1
4,2018-01-01,11.90,11.91,13.00,11.90,12.60,12.80,0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
35030,2019-12-31,13.56,15.12,15.00,14.56,15.00,15.31,1,0.0,0,12
35031,2019-12-31,13.81,13.56,15.12,15.00,14.56,15.00,1,0.0,0,12
35032,2019-12-31,14.12,13.81,13.56,15.12,15.00,14.56,1,0.0,0,12
35033,2019-12-31,14.50,14.12,13.81,13.56,15.12,15.00,1,0.0,0,12


As we can see the weather data starts from 00:00:00 whilst df starts from 02:30:00 due to missing n-5 values for previous rows. Thus we need to transform the weatherdata to match df.

In [14]:
#getting rid of the first few (5) rows that we deleted because of not having n-5 values for them
N = weather_df.shape[0] - df.shape[0]
weather_df = weather_df.iloc[N: , :]
weather_df = weather_df.reset_index()
weather_df = weather_df.drop('id', axis = 1)
weather_df


,Temperature (C),Feels like (C),Humidity (%)
0,6.1,2.0,84.33
1,6.0,1.6,80.44
2,6.0,1.6,80.44
3,5.8,1.7,82.54
4,5.8,1.7,82.54
...,...,...,...
35030,6.5,3.3,88.23
35031,6.6,4.4,87.69
35032,6.6,4.4,87.69
35033,6.7,4.1,88.13


In [15]:
df = pd.concat([df, weather_df],axis=1)
df = df.drop('From Timestamp', axis = 1)
df

,Target,n-1,n-2,n-3,n-4,n-5,dayofweek,hour,holiday,month,Temperature (C),Feels like (C),Humidity (%)
0,12.60,12.80,11.90,13.50,12.30,12.80,0,0.0,1,1,6.1,2.0,84.33
1,11.90,12.60,12.80,11.90,13.50,12.30,0,0.0,1,1,6.0,1.6,80.44
2,13.00,11.90,12.60,12.80,11.90,13.50,0,0.0,1,1,6.0,1.6,80.44
3,11.91,13.00,11.90,12.60,12.80,11.90,0,0.0,1,1,5.8,1.7,82.54
4,11.90,11.91,13.00,11.90,12.60,12.80,0,0.0,1,1,5.8,1.7,82.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35030,13.56,15.12,15.00,14.56,15.00,15.31,1,0.0,0,12,6.5,3.3,88.23
35031,13.81,13.56,15.12,15.00,14.56,15.00,1,0.0,0,12,6.6,4.4,87.69
35032,14.12,13.81,13.56,15.12,15.00,14.56,1,0.0,0,12,6.6,4.4,87.69
35033,14.50,14.12,13.81,13.56,15.12,15.00,1,0.0,0,12,6.7,4.1,88.13


# Feature engineering finished. Extract dataframe to excel

In [17]:
df.to_excel('HH_Input.xlsx', index=True, header=True)